In [ ]:
!pip install transformers accelerate safetensors torch torchvision



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch, gc
from PIL import Image
import requests

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")


Using device: cuda


In [ ]:

def recipe():
    model_id = "llava-hf/llava-1.5-7b-hf"
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    ).to(0)

    processor = AutoProcessor.from_pretrained(model_id)

    torch.cuda.empty_cache()
    gc.collect()


    image = Image.open("/content/mashed_potatoes.jpg").convert("RGB")



    user_input = "mash soft thing"


    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text":
                "You are a home-style chef. You will be shown a photo of a dish and a creatively described title.\n"
                  "Your task is to generate a simple 2–3 step recipe — just short cooking instructions, using common ingredients.\n"
                  "Keep your answer short and do not list ingredients separately — only include them within the steps.\n"
                  "Your output should be of similar tone, length and format as given in the following examples.\n\n"

                  "### Example 1:\n"
                  "Image: A burger\n"
                  "Dish title: bread stack\n"
                  "Output: Cook the patty, toast the buns, assemble with veggies, sauce, and buns.\n\n"

                  "### Example 2:\n"
                  "Image: A plate of fries\n"
                  "Dish title: golden sticks\n"
                  "Output: Peel and cut potatoes into thin sticks, deep fry, sprinkle with salt.\n\n"


                  "###Example 3:\n"
                  "Image: donuts\n"
                  "Dish title: sweet ring things\n"
                  "Output: make dough by mixing ingredients, shape into rings, fry them and add sugar and glaze\n\n"

                  "Now here’s a new one:"
                },
                {"type": "image"},
                {"type": "text", "text": f"Dish title: {user_input}. What's the recipe?"}
            ],
        },
    ]


    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


    inputs = processor(images=image, text=prompt, return_tensors='pt').to(0, torch.float16)



    with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=60)

    torch.cuda.empty_cache()
    gc.collect()


    response = processor.batch_decode(output[:, inputs["input_ids"].shape[-1]:], skip_special_tokens=True)[0]
    print(response)

In [ ]:
recipe()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Cook the potatoes, mash them with a fork, add butter and season with salt and pepper.
